In [2]:
import keras
import pickle
import cv2
import glob
import os
import numpy as np

In [3]:
UNKNOWN_IMAGES = "../data/dataset1/unknown_images/"

In [4]:
primitive_hash = list()

In [5]:
for file_path in glob.glob(UNKNOWN_IMAGES+"*.png"):
    file_name = os.path.basename(file_path)
    primitive_hash.append({"im_path":file_name})

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(primitive_hash)

In [8]:
df.head()

,im_path
0,1.png
1,10.png
2,11.png
3,12.png
4,13.png


In [9]:
test_data = {"features":[],"lables":[]}

In [10]:
def read_img(file_path:str):
    img_gray = cv2.imread(UNKNOWN_IMAGES+file_path,cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(img_gray, (128,128), interpolation = cv2.INTER_AREA)
    im_bw = cv2.threshold(resized, 127, 255, cv2.THRESH_BINARY)[1]
    return im_bw

In [11]:
test_data["features"] = df.im_path.apply(lambda x: read_img(x))

In [12]:
Y_features = []
for i in test_data['features']:
    k = i.reshape((i.shape[0], i.shape[1], 1))
    Y_features.append(k)

In [13]:
Y_features = np.stack(Y_features, axis=0)

In [14]:
Y_features.shape

(50, 128, 128, 1)

In [15]:
model = keras.models.load_model("./lenet-95")

In [16]:
y_pred = model.predict(Y_features)

In [17]:
y_pred = list(map(lambda x: 0 if x<0.5 else 1, y_pred))

In [18]:
with open('binarizer', 'rb') as handle:
    lbr = pickle.load(handle)

In [19]:
df['category'] = lbr.inverse_transform(np.array(y_pred))

In [20]:
df

,im_path,category
0,1.png,sick
1,10.png,sick
2,11.png,normal
3,12.png,sick
4,13.png,normal
5,14.png,sick
6,15.png,normal
7,16.png,sick
8,17.png,normal
9,18.png,sick


In [21]:
df.to_json("../processed/unknown_data.json")